In [8]:
import os
import pandas as pd
import torch
from tqdm import tqdm

import sys
# from models.hubert import HubertBase, HubertLarge


In [9]:
ls

00.analysis.ipynb                LICENSE
01.DataLoader.ipynb              README.md
02.OldSchoolModels.ipynb         SEMPIDataLoader.py
02.SEMPIApproach.ipynb           __init__.py
02.XAttention.ipynb              code/
10.DataFrameBuilder.ipynb        data/
AudioFeatures.ipynb              engagement_listener_speaker.csv
DataPreparation.ipynb            model_weights/


In [ ]:
import os
import torch
import pandas as pd
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, HubertModel
torchaudio.set_audio_backend("soundfile")
import soundfile as sf

def makefolder(path):
    os.makedirs(path, exist_ok=True)

def extract_and_save_features(df, feature_extractor, model, feature_dir, role):
    paths = []
    for i, row in df.iterrows():
        wav_path = row[f"{role}_audio_path"]
        if not os.path.isfile(wav_path):
            raise FileNotFoundError(f"File not found: {wav_path}")

        data, sample_rate = sf.read(wav_path)
        waveform = torch.tensor(data)

        if waveform.ndim == 2:
            waveform = waveform.mean(dim=1)

        waveform = waveform.unsqueeze(0)
        if sample_rate != 16000:
            waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)
            sample_rate = 16000

        print(f"Processing {wav_path} with sample rate {sample_rate}")
        inputs = feature_extractor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.inference_mode():
            outputs = model(**inputs)
            features = outputs.last_hidden_state

        # Fix: properly construct path and create folders
        relative_path = wav_path.split("audio/")[-1]
        no_ext = os.path.splitext(relative_path)[0]
        out_path = os.path.join(feature_dir, f"{no_ext}.pt")
        makefolder(os.path.dirname(out_path))

        print(f"Saving features to {out_path}")
        torch.save(features.squeeze(0).cpu(), out_path)
        paths.append(out_path)

    return paths

def AudioFeatures_Extractor(input_csv, output_csv, feature_dir, feature_extractor, model):
    df = pd.read_csv(input_csv)

    listener_paths = extract_and_save_features(df, feature_extractor, model, feature_dir, "listener")
    speaker_paths = extract_and_save_features(df, feature_extractor, model, feature_dir, "speaker")

    df["listener_audio_path"] = listener_paths
    df["speaker_audio_path"] = speaker_paths

    df.to_csv(output_csv, index=False)
    print(f"Saved extracted features and updated CSV to: {output_csv}")

input_csv = "./engagement_listener_speaker.csv"
output_csv = "./engagement_listener_speaker_features.csv"
feature_dir = "features/audio/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
model = HubertModel.from_pretrained("facebook/hubert-base-ls960").to(device).eval()

AudioFeatures_Extractor(input_csv, output_csv, feature_dir, feature_extractor, model)


<ipython-input-5-53c9acba9841>:82: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


Processing data/engagement/audio/Ask_a_Therapist_How_to_Manage_Mental_Health_During_a_Pandemic/clip0001/Ask_a_Therapist__How_to_Manage_Mental_Health_During_a_Pandemic2_clip1.wav with sample rate 16000
Saving features to features/audio/Ask_a_Therapist_How_to_Manage_Mental_Health_During_a_Pandemic/clip0001/Ask_a_Therapist__How_to_Manage_Mental_Health_During_a_Pandemic2_clip1.pt
Processing data/engagement/audio/Ask_a_Therapist_How_to_Manage_Mental_Health_During_a_Pandemic/clip0001/Ask_a_Therapist__How_to_Manage_Mental_Health_During_a_Pandemic2_clip2.wav with sample rate 16000
Saving features to features/audio/Ask_a_Therapist_How_to_Manage_Mental_Health_During_a_Pandemic/clip0001/Ask_a_Therapist__How_to_Manage_Mental_Health_During_a_Pandemic2_clip2.pt
Processing data/engagement/audio/Ask_a_Therapist_How_to_Manage_Mental_Health_During_a_Pandemic/clip0001/Ask_a_Therapist__How_to_Manage_Mental_Health_During_a_Pandemic2_clip4.wav with sample rate 16000
Saving features to features/audio/Ask_a_T

In [ ]:
import torchaudio
torchaudio.set_audio_backend("soundfile")
print(torchaudio.list_audio_backends()) 
print("Current backend:", torchaudio.get_audio_backend())


['soundfile']
Current backend: None


<ipython-input-1-5c8440b3e0bf>:2: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
<ipython-input-1-5c8440b3e0bf>:4: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  print("Current backend:", torchaudio.get_audio_backend())


In [ ]:
import soundfile as sf
sf.info("."
"/data/engagement/audio/August_Facebook_Live/clip0002/August_Facebook_Live2_clip1.wav")  # Should print info without error

/Users/athenasaghi/VSProjects/SEMPI-1/data/engagement/audio/August_Facebook_Live/clip0002/August_Facebook_Live2_clip1.wav
samplerate: 16000 Hz
channels: 2
duration: 8.360 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]